<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>

# Hyperparameter Tuning

## *Data Science Unit 4 Sprint 2 Assignment 4*

## Your Mission, should you choose to accept it...

To hyperparameter tune and extract every ounce of accuracy out of this telecom customer churn dataset: <https://drive.google.com/file/d/1dfbAsM9DwA7tYhInyflIpZnYs7VT-0AQ/view> 

## Requirements

- Load the data
- Clean the data if necessary (it will be)
- Create and fit a baseline Keras MLP model to the data.
- Hyperparameter tune (at least) the following parameters:
 - batch_size
 - training epochs
 - optimizer
 - learning rate (if applicable to optimizer)
 - momentum (if applicable to optimizer)
 - activation functions
 - network weight initialization
 - dropout regularization
 - number of neurons in the hidden layer
 
 You must use Grid Search and Cross Validation for your initial pass of the above hyperparameters
 
 Try and get the maximum accuracy possible out of this data! You'll save big telecoms millions! Doesn't that sound great?


In [145]:
import pandas as pd
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import train_test_split


In [157]:
df = pd.read_csv('WA_Fn-UseC_-Telco-Customer-Churn - WA_Fn-UseC_-Telco-Customer-Churn.csv')

# Dropping customer ID
df = df.drop(['customerID'], axis='columns')

df.Churn = df.Churn.replace({'No':0,'Yes':1})
df = df.dropna()

y = df['Churn']
X = df.drop(['Churn'], axis='columns')

In [158]:
df.shape

(7032, 20)

In [159]:
encoder = OrdinalEncoder()
X_enc = encoder.fit_transform(X)

x_train, x_test, y_train, y_test = train_test_split(X_enc, y)


In [160]:
# numpy.random.seed(seed)

x_train = np.array(x_train)
x_test = np.array(x_test)

y_train = np.array([y_train]).T
y_test = np.array([y_test]).T

In [149]:
x_train.shape

(5274, 19)

In [161]:
y_train.shape

(5274, 1)

In [162]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

x_train = scaler.fit_transform(x_train)
x_test = scaler.fit_transform(x_test)

In [151]:
x_test.shape

(1758, 19)

In [163]:
y_train

array([[0],
       [0],
       [0],
       ...,
       [0],
       [1],
       [0]])

In [165]:
y_train.shape

(5274, 1)

In [167]:
y_test.shape

(1758, 1)

In [168]:
x_test.shape

(1758, 19)

In [166]:
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Important Hyperparameters
inputs = x_train.shape[1]
epochs = 50
batch_size = 10


# Create Model
model = Sequential()
model.add(Dense(21, activation='relu', input_dim=19))
model.add(Dense(21, activation='relu'))
model.add(Dense(1))
# Compile Model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Fit Model

model.fit(x_train, y_train,
#           validation_data=(x_test, y_test),
          epochs=epochs,
          batch_size=epochs,
          verbose=True
         )

Train on 5274 samples
Epoch 1/50
5274/5274 [==============================] - 1s 132us/sample - loss: 3.6634 - accuracy: 0.7226
Epoch 2/50
5274/5274 [==============================] - 0s 56us/sample - loss: 1.8246 - accuracy: 0.7395
Epoch 3/50
5274/5274 [==============================] - 0s 54us/sample - loss: 1.0943 - accuracy: 0.7596
Epoch 4/50
5274/5274 [==============================] - 0s 53us/sample - loss: 0.8455 - accuracy: 0.7480
Epoch 5/50
5274/5274 [==============================] - 0s 58us/sample - loss: 0.6830 - accuracy: 0.7600
Epoch 6/50
5274/5274 [==============================] - 0s 54us/sample - loss: 0.6221 - accuracy: 0.7691
Epoch 7/50
5274/5274 [==============================] - 0s 58us/sample - loss: 0.5932 - accuracy: 0.7766
Epoch 8/50
5274/5274 [==============================] - 0s 55us/sample - loss: 0.5739 - accuracy: 0.7751
Epoch 9/50
5274/5274 [==============================] - 0s 55us/sample - loss: 0.5430 - accuracy: 0.7710
Epoch 10/50
5274/5274 [=========

In [176]:
import numpy
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier # There's a KerasRegressor too...

# fix random seed for reproducibility
seed = 777
numpy.random.seed(seed)

# # load dataset
# url ="https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.data.csv"

# dataset = pd.read_csv(url, header=None).values

# # split into input (X) and output (Y) variables
# X = dataset[:,0:8]
# Y = dataset[:,8]

X = x_train
Y = y_train

# Function to create model, required for KerasClassifier
def create_model():
    # create model
    model = Sequential()
    model.add(Dense(21, input_dim=19, activation='relu'))
#     model.add(Dense(7, activation='relu'))
#     model.add(Dense(7, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

# create model
model = KerasClassifier(build_fn=create_model, epochs=100, batch_size=10, verbose=0)

# define the grid search parameters
# batch_size = [10, 20, 40, 60, 80, 100]
# param_grid = dict(batch_size=batch_size, epochs=epochs)

# define the grid search parameters
param_grid = {'batch_size': [80],
              'epochs': [20, 50],
              
             }

# Create Grid Search
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1) # make sure n_jobs is 1 for keras classifier...
grid_result = grid.fit(X, Y)

# Report Results
print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f"Means: {mean}, Stdev: {stdev} with: {param}")

Best: 0.7957906524340311 using {'batch_size': 80, 'epochs': 20}
Means: 0.7957906524340311, Stdev: 0.00517185995080008 with: {'batch_size': 80, 'epochs': 20}
Means: 0.7925673127174377, Stdev: 0.0016310698037157122 with: {'batch_size': 80, 'epochs': 50}


In [179]:
#OPtimizing by optimizer...

import numpy
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier # There's a KerasRegressor too...

# fix random seed for reproducibility
seed = 777
numpy.random.seed(seed)

# # load dataset
# url ="https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.data.csv"

# dataset = pd.read_csv(url, header=None).values

# # split into input (X) and output (Y) variables
# X = dataset[:,0:8]
# Y = dataset[:,8]

X = x_train
Y = y_train

optimizer = 'Adam'

# Function to create model, required for KerasClassifier
def create_model():
    # create model
    model = Sequential()
    model.add(Dense(21, input_dim=19, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model

# create model
model = KerasClassifier(build_fn=create_model, epochs=80, batch_size=80, verbose=0)

# define the grid search parameters
# batch_size = [10, 20, 40, 60, 80, 100]
# param_grid = dict(batch_size=batch_size, epochs=epochs)

# define the grid search parameters
learn_rate = [0.001, 0.01, 0.1, 0.2, 0.3]
momentum = [0.0, 0.2, 0.4, 0.6, 0.8, 0.9]
param_grid = dict(learn_rate=learn_rate, momentum=momentum)

# Create Grid Search
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1, cv=3) # make sure n_jobs is 1 for keras classifier...
grid_result = grid.fit(X, Y)

# Report Results
print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f"Means: {mean}, Stdev: {stdev} with: {param}")

ValueError: learn_rate is not a legal parameter

In [ ]:
optimizer = ['SGD', 'RMSprop', 'Adagrad', 'Adadelta', 'Adam', 'Adamax', 'Nadam']
param_grid = dict(optimizer=optimizer)

In [ ]:
# From JC, how to tune learning rate: an example


import wandb
from wandb.keras import WandbCallback
wandb.init(project="boston", entity="lambda-ds6") #Initializes and Experiment

from tensorflow.keras.optimizers import Adam

# Important Hyperparameters
X =  x_train
y =  y_train

inputs = X.shape[1]
wandb.config.epochs = 100
wandb.config.batch_size = 10
wandb.config.learning_rate = .01

# Create Model
model = Sequential()
model.add(Dense(64, activation='relu', input_shape=(inputs,)))
model.add(Dense(64, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(1))

# Optimizer 
adam = Adam(learning_rate=wandb.config.learning_rate)

# Compile Model
model.compile(optimizer=adam, loss='mse', metrics=['mse', 'mae'])

# Fit Model
model.fit(X, y, 
          validation_split=0.33, 
          epochs=wandb.config.epochs, 
          batch_size=wandb.config.batch_size, 
          callbacks=[WandbCallback()]
         )

In [ ]:
# use Normalizer to transform array to 0,1
x_train = Normalizer().fit(x_train).transform(x_train)
x_test = Normalizer().fit(x_test).transform(x_test) # Hold off on running...

## Stretch Goals:

- Try to implement Random Search Hyperparameter Tuning on this dataset
- Try to implement Bayesian Optimiation tuning on this dataset using hyperas or hyperopt (if you're brave)
- Practice hyperparameter tuning other datasets that we have looked at. How high can you get MNIST? Above 99%?
- Study for the Sprint Challenge
 - Can you implement both perceptron and MLP models from scratch with forward and backpropagation?
 - Can you implement both perceptron and MLP models in keras and tune their hyperparameters with cross validation?